In [53]:
# %load day3.py
import numpy as np
from itertools import zip_longest

# Part 1
default_origin = (0, 0)

In [221]:
class Grid():
    def __init__(self, origin=default_origin):
        self.origin = origin
        self.grid = [origin, ]
        self._current_x = self.origin[0]
        self._current_y = self.origin[1]

    def show(self):
        print(self.grid)

    def parse_instruction(self, inst):
        direction = inst[0]
        magnitude = int(inst[1:])

        if direction in ('D', 'L'):
            magnitude = -1 * magnitude

        if direction in ('D', 'U'):
            direction = 'v'
        else:
            direction = 'h'

        return (magnitude, direction)

    def update(self, instruction):
        magnitude, direction = self.parse_instruction(instruction)
        if direction == "v":
            new_y = self._current_y + magnitude 
            low = min(new_y, self._current_y)
            high = max(new_y, self._current_y)
            new_coords = list(zip_longest( [self._current_x, ], range(low, high), fillvalue=self._current_x))
            self._current_y = new_y
            
        elif direction == "h":
            new_x = self._current_x + magnitude
            low = min(new_x, self._current_x)
            high = max(new_x, self._current_x)
            new_coords = list(zip_longest(range(low, high), [self._current_y, ], fillvalue=self._current_y))
            self._current_x = new_x

        for i in new_coords:
            self.grid.append(i)
        # Fix the missing corners bug with a hammer
        current_coords = (self._current_x, self._current_y)
        self.grid.append(current_coords)

    def build_path(self, instructions):
        for inst in instructions:
            self.update(inst)

In [222]:
def find_intersections(grid1, grid2):
    result = set(grid1.grid).intersection(set(grid2.grid))
    result.remove(grid1.origin)
    return result


def calc_distance(point, origin=default_origin):
    """Using Manhattan distance!"""
    deltas = np.abs(np.array(point) - np.array(origin))
    return (sum(deltas))


def find_closest_intersection(grid1, grid2):
    itxs = find_intersections(grid1, grid2)
    distances = [calc_distance(i, origin=grid1.origin) for i in itxs if (i != grid1.origin)]
    
    if len(distances) == 0:
        raise Exception("No intersections detected")

    return min(distances)


def main(l1, l2):
    l1 = l1.split(',')
    l2 = l2.split(',')

    g1 = Grid()
    g2 = Grid()

    g1.build_path(l1)
    g2.build_path(l2)

    result = find_closest_intersection(g1, g2)
    return result
    

In [230]:
# Tests

def test_main():
    i1 = 'R75,D30,R83,U83,L12,D49,R71,U7,L72'
    i2 = 'U62,R66,U55,R34,D71,R55,D58,R83'
    result = main(i1, i2)
    assert (result == 159)
    
    i1 = 'R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51'
    i2 = 'U98,R91,D20,R16,D67,R40,U7,R15,U6,R7'
    result = main(i1, i2)
    assert (result == 135)


def test_update():
    g = Grid(nrows=3, ncols=3)
    g.update(2, 'h')
    g.update(2, 'v')
    g.update(-2, 'h')
    expected_result = np.array([[1, 1, 1], [0, 0, 1], [1, 1, 1]])
    assert (g.grid == expected_result).all()


def test_parse_instructions():
    my_grid = Grid()
    assert (my_grid.parse_instruction('U10') == (10, 'v'))
    assert (my_grid.parse_instruction('D10') == (-10, 'v'))
    assert (my_grid.parse_instruction('R10') == (10, 'h'))
    assert (my_grid.parse_instruction('L10') == (-10, 'h'))

    
def test_find_intersections():
    g1 = Grid()
    g2 = Grid()
    
    g1.build_path(['U1', 'R1', 'U4', 'D1'])
    g2.build_path(['R1', 'U1', 'L3'])
    
    result = find_intersections(g1, g2)
    assert (len(list(result)) == 1)
    assert (result == {(1,1)})


def test_manhattan_distance_calculator():
    assert (calc_distance((0, 0)) == 0)
    assert (calc_distance((12, 3)) == 15)
    assert (calc_distance((0, 0), origin=(10, 10)) == 20)


def test_find_closest_intersection():
    g1 = np.zeros((3, 3))
    g2 = np.zeros((3, 3))
    g1[0, 0] = g1[1, 1] = g1[2, 2] = g1[0, 2] = 1
    g2[2, 0] = g2[1, 1] = g2[0, 2] = 1

    result = find_closest_intersection(g1, g2)
    assert (result == 2)

    result = find_closest_intersection(g1, g2, origin=np.array([0,1]))
    assert (result == 1)

In [231]:
with open('inputs.txt','r') as f:
    foo = f.readlines()

i1 = foo[0].strip()
i2 = foo[1].strip()
main(i1, i2)

3229

In [233]:
g1 = Grid()
g1.build_path(['L2', 'D5', 'U1'])

In [235]:
g1.grid

[(0, 0),
 (-2, 0),
 (-1, 0),
 (-2, 0),
 (-2, -5),
 (-2, -4),
 (-2, -3),
 (-2, -2),
 (-2, -1),
 (-2, -5),
 (-2, -5),
 (-2, -4)]